In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [17]:
!python --version

Python 3.12.12


In [2]:
df = pd.read_excel("labeled_dataset.xlsx")

/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
df.head()

,Unnamed: 0,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words4
0,0,YouTube is making clear there will be no “birt...,https://eu.usatoday.com/story/tech/2020/02/03/...,usa-today,elections-2020,center,1,1,Biased,Somewhat factual but also opinionated,YouTube says no ‘deepfakes’ or ‘birther’ video...,"['belated', 'birtherism']"
1,1,The increasingly bitter dispute between Americ...,https://www.nbcnews.com/news/sports/women-s-te...,msnbc,sport,left,1,1,Non-biased,Entirely factual,"FRISCO, Texas — The increasingly bitter disput...",['bitter']
2,2,So while there may be a humanitarian crisis dr...,https://www.alternet.org/2019/01/here-are-5-of...,alternet,immigration,left,1,1,Biased,Expresses writer’s opinion,Speaking to the country for the first time fro...,['crisis']
3,3,A professor who teaches climate change classes...,https://www.breitbart.com/politics/2019/05/09/...,breitbart,environment,right,1,1,Non-biased,No agreement,A professor who teaches climate change classes...,['legitimate']
4,4,"Looking around the United States, there is nev...",https://thefederalist.com/2020/03/11/woman-who...,federalist,abortion,right,1,1,Biased,Somewhat factual but also opinionated,The left has a thing for taking babies hostage...,"['killing', 'never', 'developing', 'humans', '..."


In [4]:
df["Label_bias"].value_counts()

,count
Label_bias,
Biased,1018
Non-biased,533
No agreement,149


In [5]:
texts = df['sentence'].astype(str).tolist()
labels = df['Label_bias'].tolist()

In [6]:
df["Label_bias"].dropna(inplace=True)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Fill NaNs in labels first
df['Label_bias'] = df['Label_bias'].fillna('Non-biased')

# Encode labels to integers
le = LabelEncoder()
df['Label_bias'] = le.fit_transform(df['Label_bias'])

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    df['sentence'].tolist(),
    df['Label_bias'].tolist(),
    test_size=0.2,
    random_state=42
)

In [8]:
y_train[0]

2

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(
    X_train,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors='tf'
)

test_encodings = tokenizer(
    X_test,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors='tf'
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(32)

In [11]:
model = TFBertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=3,  # number of unique labels
    use_safetensors=False
)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

model.compile(optimizer="adagrad", loss=loss, metrics=metrics)

tf_model.h5:   0%|          | 0.00/536M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=8
)

Epoch 1/8
43/43 [==============================] - 33s 779ms/step - loss: 0.6131 - accuracy: 0.7735 - val_loss: 0.7506 - val_accuracy: 0.6882
Epoch 2/8
43/43 [==============================] - 32s 751ms/step - loss: 0.5734 - accuracy: 0.7890 - val_loss: 0.7638 - val_accuracy: 0.6853
Epoch 3/8
43/43 [==============================] - 33s 770ms/step - loss: 0.5587 - accuracy: 0.7949 - val_loss: 0.8456 - val_accuracy: 0.6471
Epoch 4/8
43/43 [==============================] - 32s 749ms/step - loss: 0.5253 - accuracy: 0.8088 - val_loss: 0.7817 - val_accuracy: 0.6647
Epoch 5/8
43/43 [==============================] - 32s 751ms/step - loss: 0.4776 - accuracy: 0.8301 - val_loss: 0.7852 - val_accuracy: 0.6794
Epoch 6/8
43/43 [==============================] - 32s 749ms/step - loss: 0.4462 - accuracy: 0.8478 - val_loss: 0.8300 - val_accuracy: 0.6735
Epoch 7/8
43/43 [==============================] - 32s 752ms/step - loss: 0.4172 - accuracy: 0.8559 - val_loss: 0.8339 - val_accuracy: 0.6824
Epoch 

In [14]:
loss, accuracy = model.evaluate(test_dataset)
print("Test Accuracy:", accuracy)

11/11 [==============================] - 2s 205ms/step - loss: 0.8356 - accuracy: 0.6794
Test Accuracy: 0.679411768913269


In [15]:
# Save the model
model.save_pretrained("./my_bert_model")

# Save the tokenizer
tokenizer.save_pretrained("./my_bert_tokenizer")

# Save the label encoder
import joblib
joblib.dump(le, "label_encoder.pkl")

['label_encoder.pkl']